In [ ]:
from elasticsearch import Elasticsearch

In [2]:
es = Elasticsearch(
    "http://localhost:9200",
    basic_auth=("elastic", "")
)
es.ping()
# true if connection is successfull

True

## Prepare the data

In [3]:
# convert the relevant fields to vectors via BERT model
import pandas as pd
